# Preprocessing

In [ ]:
from cng.utils import *
from cng.h3 import *
from ibis import _
import os
from osgeo import gdal
from minio import Minio
import streamlit 
from datetime import timedelta
import geopandas as gpd

# Get signed URLs to access license-controlled layers
key = st.secrets["MINIO_KEY"]
secret = st.secrets["MINIO_SECRET"]
client = Minio("minio.carlboettiger.info", key, secret)

con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
endpoint = os.getenv("AWS_S3_ENDPOINT", "minio.carlboettiger.info")
duckdb_install_h3()

set_secrets(con)

#### Converting data to hexes at zoom 8

In [ ]:
def h3_from_geom(con, name, cols, zoom = 8):
    """
    Computes hexes directly from geometry.
    """
    cols = ", ".join(cols) if isinstance(cols, list) else cols
    con.raw_sql(f'''
    CREATE OR REPLACE TEMP TABLE t2 AS
    SELECT {cols},
           h3_polygon_wkt_to_cells_string(ST_Force2D(dump.geom), {zoom}) AS h{zoom}
    FROM (
        SELECT {cols}, UNNEST(ST_Dump(geom)) AS dump
        FROM {name}
    )
    ''')
    
    con.sql(f'''
        SELECT {cols}, UNNEST(h{zoom}) AS h{zoom},
        ST_GeomFromText(h3_cell_to_boundary_wkt(UNNEST(h{zoom}))) AS geom
        FROM t2
    ''').to_parquet(f"{name}_h3_z{zoom}.parquet")
    return 

# TPL Conservation Almanac

Hexing this data at zoom 8 level

In [ ]:
tpl = client.get_presigned_url(
    "GET",
    "shared-tpl",
    "tpl.parquet",
    expires=timedelta(hours=2),
)

cols = ['fid', 'TPL_ID', 'State', 'County', 'Municipality',
       'Site_Name', 'Reported_Acres', 'Close_Year', 'Close_Date', 'Owner_Name',
       'Owner_Type', 'Manager_Name', 'Manager_Type', 'Purchase_Type',
       'EasementHolder_Name', 'EasementHolder_Type', 'Public_Access_Type',
       'Purpose_Type', 'Duration_Type', 'Data_Provider', 'Data_Source',
       'Source_Date', 'Data_Aggregator', 'Comments', 'Amount', 'Program_ID',
       'Program_Name', 'Sponsor_ID', 'Sponsor_Name', 'Sponsor_Type']


tpl_table = (con.read_parquet(tpl)
             .mutate(geom = _.geom.convert("ESRI:102039", "EPSG:4326"))
            )

con.create_table('tpl', tpl_table, overwrite=True)
h3_from_geom(con, 'tpl', cols)

client.fput_object(bucket_name = "shared-tpl",
           object_name = "tpl_h3_z8.parquet",
           file_path = "tpl_h3_z8.parquet") 

# Census

Getting polygons and FIPS codes from Census state, county, place, and subdivision data. 

#### State

In [ ]:
%%time
# convert shape to parquet 
gdf = gpd.read_file('tl_2024_us_state.shp').to_crs('epsg:4326').rename_geometry('geom').rename(columns={"GEOID": "FIPS"})
url = "s3://public-census/state/2024_us_state.parquet"
con.create_table('state_wkt', gdf, overwrite=True).to_parquet(url)

# get h3
con.raw_sql("""
CREATE OR REPLACE TABLE state AS
SELECT * EXCLUDE geom,
  ST_GeomFromWKB(geom) AS geom
FROM state_wkt
""")

cols = ['NAME','FIPS']
h3_from_geom(con, 'state', cols)
state_geo = con.table('state')


In [ ]:
state_url = "https://www2.census.gov/geo/docs/reference/codes2020/national_state2020.txt"
state_fips = (con.read_csv(state_url)
              .rename(name = "STATE_NAME", FIPS = "STATEFP")
              .select("STATE","name","FIPS"))

state = state_fips.left_join(state_geo, 'FIPS')
state.to_parquet("s3://public-census/state/2024_us_state_h3_z8.parquet")

#### County

In [ ]:
%%time
# convert shape to parquet 
gdf = gpd.read_file('tl_2024_us_county.shp').to_crs('epsg:4326').rename_geometry('geom').rename(columns={"GEOID": "FIPS"})
url = "s3://public-census/county/2024_us_county.parquet"
con.create_table('county_wkt', gdf, overwrite=True).to_parquet(url)

# get h3
con.raw_sql("""
CREATE OR REPLACE TABLE county AS
SELECT * EXCLUDE geom,
  ST_GeomFromWKB(geom) AS geom
FROM county_wkt
""")

cols = ['NAMELSAD','FIPS']
h3_from_geom(con, 'county', cols)
county_geo = con.table('county')

In [ ]:
county_url = "https://www2.census.gov/geo/docs/reference/codes2020/national_county2020.txt"

county_fips = (con.read_csv(county_url)
               .rename(state = "STATE", county = "COUNTYNAME")
               .mutate(name=_.county.re_replace(county_match_pattern, "").strip())
               .mutate(FIPS = _.COUNTYFP + _.STATEFP)
               .select("state","name","FIPS"))

county = county_fips.left_join(county_geo, 'FIPS')
county.to_parquet("s3://public-census/county/2024_us_county_h3_z8.parquet")

#### Cities (places + subdivisions)

Note: Some cities are listed in both "Places" and "Subdivisions", so we will use `distinct()` to avoid duplicates.

In [ ]:
city_cols = ["state","county","FIPS","name"]

places_url = "https://www2.census.gov/geo/docs/reference/codes2020/national_place_by_county2020.txt"
places_fips = (con.read_csv(places_url)
               .rename(state = "STATE", county = "COUNTYNAME", city = "PLACENAME")
               .mutate(name=_.city.re_replace(match_pattern, "").strip())
               .mutate(FIPS = _.COUNTYFP + _.STATEFP)
               .select(city_cols))

subdivisions_url = "https://www2.census.gov/geo/docs/reference/codes2020/national_cousub2020.txt"
subdivisions_fips = (con.read_csv(subdivisions_url)
                     .rename(state = "STATE", county = "COUNTYNAME", city = "COUSUBNAME")
                     .mutate(name=_.city.re_replace(match_pattern, "").strip())
                     .mutate(FIPS = _.COUNTYFP + _.STATEFP)
                     .select(city_cols))

city_fips = places_fips.union(subdivisions_fips).distinct() #get unique -> some cities are listed in both places and subdivisions
city = city_fips.left_join(county_geo, 'FIPS')
city.to_parquet("s3://public-census/city/2024_us_city_h3_z8.parquet")

# Landvote

We want to join Landvote data with TPL Conservation Almanac, but Landvote doesn't have spatial data.

However, we can join Landvote with Census data to get FIPS codes and hexes. 
- First, need to split up landvote into its 3 jurisdictions: state, county, and municipals
- Join states with Census "states" to get state FIPS/hex
- Join counties with Census "counties" to get county FIPS/hex
- Join municipals with Census "places" and "subdivisions" to get county FIPS/hex
- Then join all municipal, county, and state data back together!



In [ ]:
landvote_csv = client.get_presigned_url(
    "GET",
    "shared-tpl",
    "landvote.csv",
    expires=timedelta(hours=2),
)

match_pattern = r"(?i)\s*(city|town|village|charter|municipality|Borough)\b"
landvote = (con.read_csv(landvote_csv, ignore_errors=True)
            .rename(jurisdiction = "Jurisdiction Type", state = "State")
            .mutate(state = _.state.substitute({'Ore':'OR'}))
            .mutate(name=_['Jurisdiction Name'].re_replace(match_pattern, "").strip())
            .mutate(id=ibis.row_number().over()))

final_columns = ['id',
    'FIPS',
    'state',
    'state_name',
    'county',
    'city',
    'jurisdiction',
    'Date',
    'Description',
    'Finance Mechanism',
    '"Other" Comment',
    'Purpose',
    'Total Funds at Stake',
    'Conservation Funds at Stake',
    'Total Funds Approved',
    'Conservation Funds Approved',
    'Pass?',
    'Status',
    '% Yes',
    '% No',
    'Notes',
    'Voted Acq. Measure',
    'geom',
    'h8']

#### State level

In [ ]:
states = (landvote.filter(_.jurisdiction == "State")
            .rename(state_name = "Jurisdiction Name")
            .mutate(county = ibis.literal('None'))
            .mutate(county_fips = ibis.literal('None'))
            .mutate(city = ibis.literal('None')))

landvote_state = (states.left_join(state_geo, [states.name.upper() == state_geo.name.upper()])
                   .select(final_columns))

#adding state ID and state name from the county/city
state_ids = state_geo.select('name','STATE').rename(state_name = 'name')

#### County level

In [ ]:
county_match_pattern = r"(?i)\s*(County)\b"

counties = (landvote.filter(_.jurisdiction == "County")
            .rename(county = "Jurisdiction Name")
            .mutate(city = ibis.literal('None'))
            .mutate(name=_.name.re_replace(county_match_pattern, "").strip()))

landvote_county = (counties.left_join(county_geo, [counties.name.upper() == county_geo.name.upper(), 
                                                    counties.state == county_geo.state])
                   .inner_join(state_ids, [counties.state == state_ids.STATE])
                   .select(final_columns))

#### Municipal level

Because there isn't a 1 to 1 match from municipals to Census data, we need to use both "Places" and "Subdivisons". 

In [ ]:
municipals = landvote.filter(_.jurisdiction == "Municipal").rename(city = "Jurisdiction Name")

landvote_city = (municipals.left_join(city_geo, [municipals.name.upper() == city_geo.name.upper(), 
                                                  municipals.state == city_geo.state])
                 .inner_join(state_ids, [municipals.state == state_ids.STATE])
                 .select(final_columns))

#### Joining all the landvote data with census
Note: `landvote_joined` has more rows than `landvote` because some cities span multiple counties. Each additional county creates a new row.

In [ ]:
landvote_joined = landvote_city.union(landvote_county).union(landvote_state)
landvote_joined.to_parquet("s3://shared-tpl/landvote_h3_z8.parquet"))